In [ ]:
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from src.inference import *
from src.misc import *
from src.models.predictors import *
from src.plots import *
from src.train import *
from pytorch_lightning.loggers import MLFlowLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
train_dataset = TensorDataset(torch.tensor(X_in[:700], dtype=torch.float32),
                              torch.tensor(Y_out[:700], dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:


mlf_logger = MLFlowLogger(experiment_name="AnomalyDetection", run_name="LSTM-Predictor")

checkpoint_callback = ModelCheckpoint(
    monitor="train_loss",
    save_top_k=1,
    mode="min",
    filename="checkpoint-{epoch:02d}-{train_loss:.2f}"
)

model = SequenceToOneLSTM(input_dim=3, hidden_dim=32, lr=1e-3)

trainer = pl.Trainer(
    max_epochs=50,
    logger=mlf_logger,
    callbacks=[checkpoint_callback],
    log_every_n_steps=10
)

trainer.fit(model, train_loader)
